<center><h1 style="font-size:300%;"><b style="color:red;">Titanic EDA and Cleaning</b></h1></center>


## Data description

**Here we have 12 columns-**

* PassengerId : ID of Passenger
* Pclass : Passenger class (1=1st,2=2nd,3=3rd)
* Survived : Survival (0=No,1=Yes)
* Sex : sex(male & female)
* Name : name of passengers
* Age : age of passengers
* Sibsp : Number of Siblings
* Parch : Number of Parents
* Ticket : passenger ticket number
* Fare : Passenger fare(British pound)
* Cabin : cabin
* Embarked : Port of Embarkation(C=Cherbourg ,Q=Queenstown, S=Southamption)

**Importing important libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import warnings as wr
wr.filterwarnings('ignore')


In [ ]:
#reading training and testing data

import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [ ]:
train.head()

 # Data Preprocessing

In [ ]:
train.describe().T.style.bar(subset=["mean"],color="#606ff2").background_gradient(
subset=["std"],cmap="PuBu").background_gradient(subset=["50%"],cmap="PuBu")

In [ ]:
train.info()

In [ ]:
#count NA values
train.isnull().sum()

* here we are droping passengerId bcoz there is no need for prediction
* we droping Cabin bcoz there is 80% NaN value and ticket also
* Age & Embarked have less missing value so we simply fill it using pandas fillna

In [ ]:
train=train.drop(["PassengerId","Ticket","Cabin"],axis=1)
train.head()

In [ ]:
train.shape


* now we have 8 features and one(survived) outcome feature.
* 891 total number of record
 

In [ ]:
# filling NaN values using pandas Fillna

train["Age"].fillna(train["Age"].mean(),inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0],inplace=True)

# Exploratory-Data-Analysis

# Survived column

In [ ]:
print(f'Number of people dead as 0 are {train.Survived.value_counts()[0]} and Number of people survived as 1 are {train.Survived.value_counts()[1]}')
sns.countplot(train["Survived"])
ax = plt.axes()
ax.set_facecolor("green")

plt.show()

 * It seem data is balanced there are 342 people survived and 549 is dead
 * This is over outcome feature we need to predict on testing data 

# Sex column

In [ ]:
#visualisation how many pasanger survived and how many dead 
#here we make a function for visulisation bar chart 

def bar_chart(column):
    survived=train[train["Survived"]==1][column].value_counts()
    dead=train[train["Survived"]==0][column].value_counts()
    df1=pd.DataFrame([survived,dead])
    df1.index=["Survived","Dead"]
    df1.plot(kind="bar",figsize=(10,5))
    ax = plt.axes()
    ax.set_facecolor("green")

In [ ]:
#here we can see relation between sex and Survived

bar_chart("Sex")

**by this visualisation we can understand that females have more chance to survived and male have less chance to Survived**

* Now we going to encode featue sex in Rank 0 for male and 1 for female

In [ ]:

train.loc[train['Sex'] == "male", 'Sex'] = 0
train.loc[train['Sex'] == "female", 'Sex'] = 1

In [ ]:
print(f'Number of people having sex as male are {train.Sex.value_counts()[0]} and Number of people having sex as female are {train.Sex.value_counts()[1]}')
sns.countplot(train["Sex"])
plt.show()

* Now we can say populaton of male is graterthan as female
* females have more chance to survived 
* males have less chance to survived

# Pclass column


In [ ]:
#by this visualisation we can see relation between pasenger class and Survived

bar_chart("Pclass")

* here we can see that 1st class Passenger have more chance to Survived
* 3ed class passanger have more chance to Dead** 

In [ ]:
print(f'Number of people travel in 1st class are {train.Pclass.value_counts()[1]} and Number of people travil in 3rd class are {train.Pclass.value_counts()[3]}')
sns.countplot(train["Pclass"])
plt.show()

* Now we can say 3rd class have more passenger 491
* 1st & 2nd class have same passenger 216 & 190


# SibSp column

In [ ]:

#by this visualisation we can see relation between pasenger SibSp and Survived

bar_chart("SibSp")
    

People have 0 Siblings have more chance to dead

**here we make another function to visualisation  boxplot to see the outliers**
by boxplot we see data visualisation distribution in 1st 3ed & 3 quatile and we also understand relation between Fare and Survived and we can find out outliers . You can see top right have a outliers 

In [ ]:

def box_plot(column):
    train.boxplot(by="Survived",column=[column],grid=True)
    ax = plt.axes()
    ax.set_facecolor("green")


In [ ]:
#counting total values in SibSp
print(train["SibSp"].value_counts())

box_plot("SibSp")

Here we can see 90% people have 0 to 2 siblings rest are we considerd as outlier

In [ ]:
#here we chacking largest values row on column Sibsp
train.nlargest(12,["SibSp"])

In [ ]:
#now we gpoing to remove outliers
train=train.drop([159,180,201,324,792,846,863])
train.shape

# Parch column

In [ ]:
#by this visualisation we can see relation between pasenger parch and Survived
bar_chart("Parch")

In [ ]:
#counting total values in parch
print(train["Parch"].value_counts())
box_plot("Parch")

* So we can say people have 0 parents have more chance to dead
* 95 % passenger have 0 ,1 or 2 parents rest are we considerd as outlier
Now we are going to remove those outliers 

In [ ]:
#here we chacking largest values row on column parch
train.nlargest(6,["Parch"])

In [ ]:
#now we gpoing to remove outliers
train=train.drop([678,13,25,610,638,885])
train.shape

In [ ]:
#here wr checking outliers 
f,ax=plt.subplots()
sns.violinplot(data=train.iloc[:,5:7])
sns.despine(offset=10,trim=True)


# Fare column

In [ ]:
#here we drow boxplot between fare and Survived
box_plot("Fare")

here you can see there are lot of otliers(above 200) in fare 

In [ ]:
#here we going to ploat scaterplot to see data distirbution
sns.relplot(x="Pclass",y="Fare",hue="Survived",data=train);


**here we use histogram .A histogram displays the shape and spread of continuous sample data.**

In [ ]:
def hist(column):
    train[column].plot(kind = 'hist',bins = 200,figsize = (6,6))
    plt.title(column)
    plt.xlabel(column)
    plt.ylabel("Frequency")
    ax = plt.axes()
    ax.set_facecolor("black")
    plt.show()

In [ ]:
hist("Fare")

### Bubbled Scatter

In [ ]:
fig=px.scatter(train,x="Age",y="Survived",size="Fare",color="Fare",log_x=True,size_max=50,
              template="plotly_dark",title="Age vs survived")
fig.show()

## 3-D Scatter

In [ ]:
fig=px.scatter_3d(train,x="Age",y="Survived",z="Fare",color="Fare",log_x=True,size_max=60,
              template="plotly_dark",title="Age vs survived vs Fare")
fig.show()

# Age Column

In [ ]:
#here we drow boxplot between Age and Survived
box_plot("Age")

**by this boxplot we can see that lees than 10 year old people have more chance to dead and between 20 to 35 have chance to survived**

In [ ]:
#here we going to ploat scaterplot to see data distirbution
sns.relplot(x="Pclass",y="Age",hue="Survived",data=train);

In [ ]:
#making histogram on Age
hist("Age")

**by this graph we can see Frequency of age. here we findout that more people are between  25 to 35 year old**

In [ ]:
# here we use scater plot to display data  
    
train[train.Survived ==1].plot(kind='scatter', x='Age', y='Fare',color="red")
plt.xlabel("Age")
plt.ylabel("Fare")
plt.title("Survived ==1")
plt.grid(True)
ax = plt.axes()
ax.set_facecolor("black")
plt.show()

**here we essaly say that in top of chart have a outliers. we also see data destrebution**

# Embarked column

In [ ]:
#by this visualisation we can see relation between pasenger Embarked and Survived
bar_chart("Embarked")

* by the above chart we can say that there are more chance to survived for those who bord from Southamption

* Passenger traveilling from Cherbourg port survived more than other port passenger

In [ ]:
#counting total values in Embarked
print(train['Embarked'].value_counts())

* more passenger(634) bord from Southamption
* less passenger(76) bord from Queenstown

In [ ]:
#here we going to ploat scaterplot to see data distirbution
sns.relplot(x="Embarked",y="Fare",hue="Survived",data=train);

In [ ]:
#Here we encode Embarked in Rank
train.loc[train['Embarked'] == "C", 'Embarked'] = 0
train.loc[train['Embarked'] == "Q", 'Embarked'] = 1
train.loc[train['Embarked'] == "S", 'Embarked'] = 2

# Name column

**making title feature using Name**

In [ ]:
train["Title"]=train["Name"].str.split(',',expand=True)[1].str.split('.',expand=True)[0]
train["Title"].unique()

**now we can replace many titles with a more comman name as Rare**

In [ ]:
train["Title"]=train["Title"].replace([" Don"," Rev"," Dr"," Major"," Lady"," Sir"," Col"," Capt"," the Countess"," Jonkheer"],"Rare")
train["Title"]=train["Title"].replace([" Mlle", " Ms"," Dona"]," Miss")
train["Title"]=train["Title"].replace([" Mme"," Mrs"]," Mr")
train["Title"].unique()


In [ ]:
#Here we encode title in Rank
mapping={' Mr':0, ' Miss':1, ' Master':2, 'Rare':3}
train["Title"]=train["Title"].map(mapping)

In [ ]:
bar_chart("Title")

In [ ]:
#counting total values in Title
print(train["Title"].value_counts())

In [ ]:
train.head()

In [ ]:
train=train.drop(["Name"],axis=1)
train.head()

In [ ]:
#here we going to ploat scaterplot to see data distirbution
sns.relplot(x="Title",y="Fare",hue="Survived",data=train);

### pair plot

In [ ]:
#ploting pair plot
plt.figure(figsize=(10, 10))
g=sns.PairGrid(train,hue="Survived")
g.map_offdiag(sns.scatterplot)
g.add_legend
plt.show()

In [ ]:
#checking feature dependance on outcome

plt.figure(figsize=(10, 10))
sns.heatmap(train.corr(), annot=True, linewidths=0.05, fmt= '.2f',cmap="magma")
plt.show()

here we easily findout that which features are important to survived . we also see data dependance

In [ ]:

df=train.drop(["Age","Fare"],axis=1)
df.head()

# Frome EDA we got

* Cabin column have lots of null values so we drop it,
* Ticket and paddenger Id is not usefull and does not have impact on survivl so drop it.
* Passenger travelling in higher class have more chance to survived
* Females survived more then Males.
* In the 1st class Females were more then Males it is also a resion that females have more chance to survived.
* Passenger travelling with siblings ,parents have more chance to survived.
* Passenger traveilling from Cherbourg port survived more than other port passenger.

## Please do a upvote if you like it

### Thank you